# Plotting eQTLs, increase font sizes

### Kynon Jade Benjamin and Apuã Paquola

In [ ]:
import re
import functools
import subprocess
import numpy as np
import pandas as pd
from plotnine import *
from pandas_plink import read_plink
from warnings import filterwarnings
from matplotlib.cbook import mplDeprecation

filterwarnings("ignore",category=mplDeprecation)
filterwarnings('ignore', category=UserWarning, module='plotnine.*')
filterwarnings('ignore', category=DeprecationWarning, module='plotnine.*')

## Configuration

In [ ]:
feature = "junctions"; tissue = "caudate"
config = {
    'biomart_file': '../_h/biomart.csv',
    'residual_expression_file': "/ceph/projects/v4_phase3_paper/analysis/eqtl_analysis/all/%s/" % feature+\
    "expression_gct/covariates/residualized_expression/_m/%s_residualized_expression.csv" % feature,
    'residual_expression_file2': "/ceph/projects/v4_phase3_paper/analysis/eqtl_analysis/all/%s/" % feature+\
    "expression_gct/prepare_expression/_m/%s.expression.bed.gz" % feature,
    'phenotype_file': '/ceph/projects/v4_phase3_paper/inputs/phenotypes/_m/merged_phenotypes.csv',
    'plink_file_prefix': '/ceph/projects/v4_phase3_paper/inputs/genotypes/_m/LIBD_Brain_TopMed',
    'eqtl_output_file': '/ceph/projects/v4_phase3_paper/analysis/eqtl_analysis/all/%s/' % feature +\
    'expression_gct/prepare_expression/fastqtl_permutation/_m/Brainseq_LIBD.genes.txt.gz',
    'gwas_snp_file': '../../summary_table/_m/Brainseq_LIBD_caudate_4features_PGC2.signifpairs.txt.gz'
}

## Functions

In [ ]:
@functools.lru_cache()
def feature_map(feature):
    return {"genes": "Gene", "transcripts": "Transcript", 
            "exons": "Exon", "junctions": "Junction"}[feature]

### Expression functions

In [ ]:
@functools.lru_cache()
def get_biomart_df():
    biomart = pd.read_csv(config['biomart_file'], index_col=0)
    biomart['description'] = biomart['description'].str.replace('\[Source.*$','', regex=True)
    return biomart


@functools.lru_cache()
def get_residual_expression_df():
    return pd.read_csv(config['residual_expression_file'], index_col=0).transpose()


@functools.lru_cache()
def get_residual_expression_df_v2():
    return pd.read_csv(config["residual_expression_file2"], sep='\t', index_col=3).drop(["#chr", "start", "end"], axis=1).transpose()


@functools.lru_cache()
def get_pheno_df():
    pheno_df = pd.read_csv(config['phenotype_file']).set_index("BrNum").loc[:, ["RNum", "Sex", "Dx", "Region"]]
    return pheno_df[(pheno_df["Region"] == "Caudate")].drop("Region", axis=1)


@functools.lru_cache()
def get_expression_and_pheno_df():
    return pd.merge(get_pheno_df(), get_residual_expression_df(),
                    left_index=True, right_index=True)


@functools.lru_cache()
def get_gene_id_df():
    return pd.DataFrame({'gene_id': get_residual_expression_df().columns,
                         'ensembl_gene_id': get_residual_expression_df().columns.str.replace('\..+$','', regex=True)})


@functools.lru_cache()
def gene_info_from_symbol(gene_symbol):
    return get_biomart_df()[get_biomart_df()['external_gene_name']==gene_symbol]\
        .merge(get_gene_id_df(), on='ensembl_gene_id', how='left')


@functools.lru_cache()
def gene_id_from_symbol(gene_symbol):
    df = gene_info_from_symbol(gene_symbol)
    assert df.shape[0] == 1
    return df[['gene_id']].iloc[0].values[0]

### DRD2 functions

In [ ]:
def get_drd2_junction_annotation(junction_id):
    return {
        'chr11:113424683-113474229(-)': "DRD2 junction 1L-2",
        "chr11:113424683-113475075(-)": "DRD2 junction 1-2",
        "chr11:113418137-113424366(-)": "DRD2 junction 2-3",
        "chr11:113417000-113418026(-)": "DRD2 junction 3-4",
        "chr11:113415612-113416862(-)": "DRD2 junction 4-5",
        "chr11:113414462-113415420(-)": "DRD2 junction 5-6",
        "chr11:113412884-113415420(-)": "DRD2 junction 5-7",
        "chr11:113412884-113414374(-)": "DRD2 junction 6-7",
        "chr11:113410921-113412555(-)": "DRD2 junction 7-8",}[junction_id]


def get_drd2_junctions():
    with subprocess.Popen("""cat <(head -1 /ceph/projects/v4_phase3_paper/analysis/differential_expression/_m/junctions/diffExpr_szVctl_full.txt) \
    <(grep -i drd2 /ceph/projects/v4_phase3_paper/analysis/differential_expression/_m/junctions/diffExpr_szVctl_full.txt)""", 
                          shell=True, stdout=subprocess.PIPE) as p:
        drd2_df = pd.read_csv(p.stdout, sep='\t', index_col=0)
    return drd2_df.reset_index().rename(columns={"index": "Feature"})


def get_drd2():
    drdj = get_drd2_junctions()[get_drd2_junctions()["gencodeTx"].str.contains("ENST00000362072.7|ENST00000346454.7")]
    return get_eqtl_df()[(get_eqtl_df()["gene_id"].isin(drdj.Feature))]

### Genotype and eQTL functions

In [ ]:
def letter_snp(number, a0, a1):
    '''
    Example:
    letter_snp(0, 'A', 'G') is 'AA'
    letter_snp(1, 'A', 'G') is 'AG'
    letter_snp(2, 'A', 'G') is 'GG'
    '''
    if np.isnan(number):
        return np.nan
    if len(a0)==1 and len(a1)==1:
        sep = ''
    else:
        sep = ' '
    return sep.join(sorted([a0]*int(number) + [a1]*(2-int(number))))


@functools.lru_cache()
def get_plink_tuple():
    '''
    Usage: (bim, fam, bed) = get_plink_tuple()
    '''
    return read_plink(config['plink_file_prefix'])


@functools.lru_cache()
def get_eqtl_df():
    """
    This function loads permutation eQTL analysis.
    """
    return pd.read_csv(config["eqtl_output_file"], sep='\t')


@functools.lru_cache()
def get_gwas_snps():
    """
    This function load the GWAS SNPs that have been mapped to BrainSeq.
    """
    gwas_df = pd.read_csv(config['gwas_snp_file'], sep='\t', index_col=0)
    return gwas_df[(gwas_df["Type"] == feature_map(feature))]


@functools.lru_cache()
def get_risk_allele(snp_id):
    gwas_snp = get_gwas_snp(snp_id)
    if gwas_snp['OR'].iloc[0] > 1:
        ra = gwas_snp['A1'].iloc[0]
    else:
        ra = gwas_snp['A2'].iloc[0]
    return ra


@functools.lru_cache()
def subset_bed():
    """
    This subsets the bed and bim file and returns the new subsetted
    data with shared brain_ids.
    
    This is to speed up accessing the bed file.
    """
    (bim, fam, bed) = get_plink_tuple()
    brain_ids = list(set(get_expression_and_pheno_df().index).intersection(set(fam['fid'])))
    fam_pos = list(fam[(fam["fid"].isin(brain_ids))].drop_duplicates(subset="fid").loc[:, 'i'])
    unique_snps0 = get_eqtl_df().variant_id.unique()
    unique_snps1 = get_gwas_snps()[get_gwas_snps()["gene_id"].isin(get_drd2().gene_id)].reset_index().variant_id.unique()
    unique_snps = list(set(unique_snps0).union(set(unique_snps1)))
    snp_info = bim[(bim["snp"].isin(unique_snps))].copy()
    snp_pos = list(snp_info.loc[:, "i"])
    new_bed = bed[snp_pos].compute()[:,fam_pos]
    new_bim = bim[(bim["i"].isin(snp_pos))].reset_index(drop=True)
    new_bim['ii'] = new_bim.index
    return new_bed, new_bim, brain_ids


@functools.lru_cache()
def get_snp_df(snp_id):
    '''
    Returns a dataframe containing the genotype on snp snp_id.
    The allele count is the same as in the plink files.
    
    Example: 
    get_snp_df('rs653953').head(5)
    
            rs653953_num rs653953_letter rs653953
    Br5168             0              GG    0\nGG
    Br2582             1              AG    1\nAG
    Br2378             1              AG    1\nAG
    Br5155             2              AA    2\nAA
    Br5182             2              AA    2\nAA
    '''
    bed, bim, brain_ids = subset_bed()
    snp_info = bim[bim['snp']==snp_id]
    snp_pos = snp_info.iloc[0]['ii']
    dfsnp = pd.DataFrame(bed[[snp_pos]], columns=brain_ids, index=[snp_id + '_num']).transpose().dropna()
    my_letter_snp = functools.partial(letter_snp, a0=snp_info.iloc[0]['a0'], a1=snp_info.iloc[0]['a1'])
    # the 2 - in next line is to workaround a possible bug in pandas_plink? a1 and a0 inverted
    dfsnp[[snp_id + '_num']] = 2 - dfsnp[[snp_id + '_num']].astype('int')
    dfsnp[snp_id + '_letter'] = dfsnp[snp_id + '_num'].apply(my_letter_snp)
    dfsnp[snp_id] = (dfsnp[snp_id + '_num'].astype('str') + '\n' + 
                     dfsnp[snp_id + '_letter'].astype('str')).astype('category')
    return dfsnp


@functools.lru_cache()
def get_gwas_ordered_snp_df(snp_id):
    '''
    Returns a dataframe containing the genotype on snp snp_id.
    The allele count is the number of risk alleles according to GWAS.
    
    Example: 
    get_gwas_ordered_snp_df('rs653953', 'genes').head(5)
    
            rs653953_num rs653953_letter rs653953
    Br5168             2              GG    2\nGG
    Br2582             1              AG    1\nAG
    Br2378             1              AG    1\nAG
    Br5155             0              AA    0\nAA
    Br5182             0              AA    0\nAA
    '''
    pgc = get_gwas_snps()
    dfsnp = get_snp_df(snp_id).copy()
    gwas_snp = get_gwas_snp(snp_id)
    if gwas_snp['pgc2_a1_same_as_our_counted'].iloc[0]:
        if gwas_snp['OR'].iloc[0] > 1:
            pass
        else:
            dfsnp[[snp_id + '_num']] = 2 - dfsnp[[snp_id + '_num']]
    else:
        if gwas_snp['OR'].iloc[0] > 1:
            dfsnp[[snp_id + '_num']] = 2 - dfsnp[[snp_id + '_num']]
        else:
            pass
    dfsnp[snp_id] = (dfsnp[snp_id + '_num'].astype('str') + '\n' + 
                     dfsnp[snp_id + '_letter'].astype('str')).astype('category')
    return dfsnp


### Plotting functions

In [ ]:
def get_snp_gene_pheno_df(snp_id, gene_id, snp_df_func):
    pheno_columns = list(get_pheno_df().columns)
    expr_df = get_expression_and_pheno_df()[pheno_columns + [gene_id]]
    snp_df =  snp_df_func(snp_id)
    return expr_df.merge(snp_df, left_index=True, right_index=True)
    

def simple_snp_expression_plot_impl(snp_id, gene_id, snp_df_func):
    df = get_snp_gene_pheno_df(snp_id, gene_id, snp_df_func)
    y0 = df[gene_id].quantile(.01) - 0.26
    y1 = df[gene_id].quantile(.99) + 0.26
    p = ggplot(df, aes(x=snp_id, y=gene_id)) \
    + geom_boxplot(fill="red", alpha=0.4, outlier_alpha=0) \
    + geom_jitter(width=0.25, stroke=0, alpha=0.6) \
    + ylim(y0, y1) + theme_bw(base_size=15) \
    + theme(panel_grid=element_blank(), 
            axis_title=element_text(face="bold"))
    return p
    

def simple_snp_expression_plot(snp_id, gene_id):
    return simple_snp_expression_plot_impl(snp_id, gene_id, get_snp_df)


def simple_gwas_ordered_snp_expression_plot(snp_id, gene_id):
    return simple_snp_expression_plot_impl(snp_id, gene_id, get_gwas_ordered_snp_df)


def get_gene_symbol(gene_id, biomart=get_biomart_df()):
    ensge = re.sub('\..+$','', gene_id)
    ggg = biomart[biomart['ensembl_gene_id']==ensge]
    if ggg.shape[0]==0:
        return '', ''
    gs = ggg['external_gene_name'].values[0]
    de = ggg['description'].values[0]
    if type(de)!=str:
        de = ''
    de = re.sub('\[Source:.*$','',de)
    return gs, de


def get_gwas_snp(snp_id):
    gwas = get_gwas_snps().reset_index()
    r = gwas[gwas['variant_id']==snp_id]
    assert len(r) == 1
    return r


def gwas_annotation(snp_id):
    return 'SZ GWAS pvalue: %.1e' % get_gwas_snp(snp_id).iloc[0]['P']


def eqtl_annotation(snp_id, gene_id):
    r = get_eqtl_df()[(get_eqtl_df()['variant_id']==snp_id) & 
                      (get_eqtl_df()['gene_id']==gene_id)]
    try:
        assert len(r)==1
        return 'eQTL q-value: %.1e' % r.iloc[0]['qval']
    except AssertionError:
        eqtl_df = get_gwas_snps().reset_index()
        r = eqtl_df[(eqtl_df['variant_id']==snp_id) & (eqtl_df['gene_id']==gene_id)]
        return 'eQTL nominal p-value: %.1e' % r.iloc[0]['pval_nominal']
    


def risk_allele_annotation(snp_id):
    return 'SZ risk allele: %s' % get_risk_allele(snp_id)


def annotated_eqtl_plot(snp_id, gene_id, title_prefix=[]):
    p = simple_snp_expression_plot(snp_id, gene_id)
    gene_symbol, gene_description = get_gene_symbol(gene_id)
    title ="\n".join(title_prefix + [eqtl_annotation(snp_id, gene_id)])
    p += ggtitle(title) + ylab('Residualized Expression') 
    return p


def gwas_annotated_eqtl_plot(snp_id, gene_id, title_prefix=[]):
    p = simple_gwas_ordered_snp_expression_plot(snp_id, gene_id)
    gene_symbol, gene_description = get_gene_symbol(gene_id)
    title ="\n".join(title_prefix + [eqtl_annotation(snp_id, gene_id),
                     gwas_annotation(snp_id),
                     risk_allele_annotation(snp_id)
                     ])
    p += ggtitle(title) + ylab('Residualized Expression') 
    return p


def save_plot(p, fn):
    for ext in ['png', 'pdf', 'svg']:
        p.save(fn + '.' + ext)
    

## Plot eQTLs

### DRD2

In [ ]:
drd2_df = get_drd2()

In [ ]:
for x in drd2_df.reset_index(drop=True).itertuples():
    en = get_drd2_junction_annotation(x.gene_id).replace(' ','_').replace("-", "_")
    anno = get_drd2_junction_annotation(x.gene_id)
    filename = "drd2_eqtl_{}".format(en)
    p = annotated_eqtl_plot(x.variant_id, x.gene_id, [anno, x.gene_id])
    print(filename, x.Index, x.variant_id, x.gene_id, anno)
    print(p)
    save_plot(p, filename)

### Top 5 eQTL with GWAS significant index SNP

In [ ]:
drd2_gwas_eqtl_df = get_gwas_snps().reset_index().merge(drd2_df.loc[:, ["gene_id", "qval"]], on=["gene_id"]).groupby(["gene_id", "is_index_snp"]).first().reset_index()
drd2_gwas_eqtl_df

In [ ]:
for x in drd2_gwas_eqtl_df[(drd2_gwas_eqtl_df["is_index_snp"])].itertuples():
    en = get_drd2_junction_annotation(x.gene_id).replace(' ','_').replace("-", "_")
    anno = get_drd2_junction_annotation(x.gene_id)
    filename = "drd2_eqtl_gwas_index_snp_{}".format(en)
    p = gwas_annotated_eqtl_plot(x.variant_id, x.gene_id, [anno, x.gene_id])
    print(filename, x.Index, x.variant_id, x.gene_id, anno)
    print(p)
    save_plot(p, filename)

In [ ]:
for x in drd2_gwas_eqtl_df[~(drd2_gwas_eqtl_df["is_index_snp"])].itertuples():
    en = get_drd2_junction_annotation(x.gene_id).replace(' ','_').replace("-", "_")
    anno = get_drd2_junction_annotation(x.gene_id)
    filename = "drd2_eqtl_gwas_{}".format(en)
    p = gwas_annotated_eqtl_plot(x.variant_id, x.gene_id, [anno, x.gene_id])
    print(filename, x.Index, x.variant_id, x.gene_id, anno)
    print(p)
    save_plot(p, filename)